ref. https://www.codewars.com/kata/54e320dcebe1e583250008fd (5 kyu) - 2019-12-04

Coding decimal numbers with factorials is a way of writing out numbers in a base system that depends on factorials, rather than powers of numbers.

In this system, the last digit is always 0 and is in base 0!.
The digit before that is either 0 or 1 and is in base 1!. The digit before that is either 0, 1, or 2 and is in base 2!.

More generally, the nth-to-last digit is always 0, 1, 2, ..., or n and is in base n!.
Example :

decimal number 463 is coded as "341010":
* $463 (base 10) = 3×5! + 4×4! + 1×3! + 0×2! + 1×1! + 0×0!$

If we are limited to digits 0...9 the biggest number we can code is $10! - 1$.

So we extend 0..9 with letters A to Z. With these 36 digits we can code up to:
* $36! − 1 = 37199332678990121746799944815083519999999910\space(base 10)$

We will code two functions:   
1. The first one will code a decimal number and return a string with the factorial representation : `dec2fact_str(nb)`
2. The second one will decode a string with a factorial representation and produce the decimal representation : `fact_str2dec(str)`.

Given numbers will be positive.  
Note:  
You have tests with Big Integers in Clojure, Julia, Python, Ruby, Haskell, Ocaml, Racket but not with Java and others where the number "nb" in "`dec2fact_str(nb)`" is at most a long.

Ref: http://en.wikipedia.org/wiki/Factorial_number_system

In [1]:
Base.factorial(10) - 1

3628799

In [2]:
f = 0
for i in 1:9
    f += i * Base.factorial(i)
end
f

# 9 . 9! + 8 . 8! + ... + 1 . 1! + 0 . 0!

3628799

In [3]:
using Test
# using BenchmarkTools

In [4]:
const C21 = 21  # Int64  

function myfact(n::T) where T <: Union{Int64, BigInt}
    (n == 0::T || n == 1::T) && return 1::T

    n < C21 && return _fact(n)
        
   # use BigInt
   f = _fact(20)
   return _fact(n, BigInt(f))
end

function _fact(n::T) where T <: Integer
    @assert n < C21
    f = 1
    for i in 2:n
        f *= i
    end
    f
end

function _fact(n::T1, f::T) where {T1 <: Integer,  T <: BigInt}
    for i in T(C21):T(n)
        f *= BigInt(i)
    end
    f
end


_fact (generic function with 2 methods)

In [5]:
@code_warntype myfact(5)

Variables
  #self#::Core.Compiler.Const(myfact, false)
  n::Int64
  f::Int64
  @_4::Bool

Body::Union{Int64, BigInt}
1 ─       Core.NewvarNode(:(f))
│   %2  = Core.typeassert(0, $(Expr(:static_parameter, 1)))::Core.Compiler.Const(0, false)
│   %3  = (n == %2)::Bool
└──       goto #3 if not %3
2 ─       (@_4 = %3)
└──       goto #4
3 ─ %7  = Core.typeassert(1, $(Expr(:static_parameter, 1)))::Core.Compiler.Const(1, false)
└──       (@_4 = n == %7)
4 ┄       goto #6 if not @_4
5 ─ %10 = Core.typeassert(1, $(Expr(:static_parameter, 1)))::Core.Compiler.Const(1, false)
└──       return %10
6 ─ %12 = (n < Main.C21)::Bool
└──       goto #8 if not %12
7 ─ %14 = Main._fact(n)::Int64
└──       return %14
8 ─       (f = Main._fact(20))
│   %17 = Main.BigInt(f)::BigInt
│   %18 = Main._fact(n, %17)::BigInt
└──       return %18


In [6]:
@test myfact(0) == 1
@test myfact(1) == 1
@test myfact(5) == 120
@test myfact(6) == 720

@test myfact(10) == 3_628_800
@test myfact(20) == Base.factorial(big(20))
@test myfact(21) == Base.factorial(big(21))
@test myfact(22) == Base.factorial(big(22))

@test myfact(25) == Base.factorial(big(25)) # 7_034_535_277_573_963_776
@test myfact(36) == Base.factorial(big(36))
@test myfact(38) == Base.factorial(big(38))
@test myfact(50) == Base.factorial(big(50))

Test Passed

In [7]:
myfact(50)

30414093201713378043612608166064768844377641568960512000000000000

In [8]:
#        1    2    3    4    5    6    7    8    9    10   11
BASE = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "A", "B", "C", "D", "E", "F", "G", "H", "I",
"J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z"]

function dec2fact_str(n::T) where T <: Union{Int64, BigInt}
    div = one(eltype(n))
    s = ""
    t = eltype(n)
    while true
        q, r = n ÷ div, n % div
        s = "$(BASE[r+1])$(s)"
        n = q
        div += 1
        n == zero(t) && (return s)     
    end
    # throw exception?
end

dec2fact_str (generic function with 1 method)

In [9]:
dec2fact_str(463)

"341010"

In [10]:
dec2fact_str(5640)

"10500000"

In [11]:
dec2fact_str(4790016000)

"A000000000000"

In [12]:
dec2fact_str(101129453440100)

"4D505603120203100"

In [13]:
# dec2fact_str(37199332678990121746799944815083519999999910)

In [14]:
REV_BASE = Dict(zip(BASE, [i for i in 0:36])) 
REV_BASE["A"]

10

In [15]:
function fact_str2dec(s::String)::Union{Int64, BigInt}
    n = length(s) - 1
    ary = split(s, "")
    f = BigInt(REV_BASE[ary[1]] * n)
    for d in ary[2:end]
        n -= 1
        if n > 0
            f = (f + REV_BASE[d]) * n
        else
            f += REV_BASE[d]
        end
    end
    return f
end

fact_str2dec (generic function with 1 method)

In [16]:
fact_str2dec("341010")

463

In [17]:
fact_str2dec("A000000000000")

4790016000

In [18]:
fact_str2dec("4D505603120203100")

101129453440100

In [19]:
dec2fact_str(5640)

"10500000"

In [20]:
@test fact_str2dec(dec2fact_str(463)) == 463
@test fact_str2dec(dec2fact_str(4790016000)) == 4790016000
@test fact_str2dec(dec2fact_str(5640)) == 5640
@test fact_str2dec(dec2fact_str(101129453440100)) == 101129453440100

Test Passed

In [21]:
@test dec2fact_str(fact_str2dec("341010")) == "341010"
@test dec2fact_str(fact_str2dec("A000000000000")) == "A000000000000"
@test dec2fact_str(fact_str2dec("4D505603120203100")) == "4D505603120203100"
@test dec2fact_str(fact_str2dec("10500000")) == "10500000"


Test Passed